In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras.applications import DenseNet201
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
from tqdm import tqdm
import gc
import cv2

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [4]:
img_width, img_height = (224, 224)

train_data_dir = './data/train_img/'
test_data_dir = './data/test_img/'
epochs = 10
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [5]:
densenet_201 = DenseNet201(include_top=False, input_shape=input_shape, weights='imagenet')

In [6]:
def get_model():
# add a global spatial average pooling layer
    x = densenet_201.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=densenet_201.input, outputs=predictions)
    
    return model

In [7]:
model = get_model()

We'll start with training the head(last layer) only as that layer is initialized randomaly and we don't want to affect the other layers weights as while backpropogation. 

In [8]:
#train only last layer
for layer in model.layers[:-1]:
    layer.trainable = False

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [9]:
def augment_brightness(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    random_bright = .25 + np.random.uniform()
    img[:,:,2] = img[:,:,2] * random_bright
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(preprocessing_function=augment_brightness,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True, 
                                   rotation_range = 20,
                                   channel_shift_range=0.2,
#                                    zca_whitening=True
                                   )

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv')
attributes = pd.read_csv('./data/attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('./data/classes.txt', delimiter='\t', header=None, index_col=0)

In [24]:
def get_imgs(src, df, labels = False):
    if labels == False:
        imgs = []    
        files = df['Image_name'].values
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
        return np.array(imgs)
    else:
        imgs = []
        labels = []
        files = os.listdir(src)
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
            labels.append(df.loc[file].values)
        return np.array(imgs), np.array(labels)

In [11]:
import pickle

with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
X_tra = data['train_x']
y_tra = data['train_y']
X_val = data['valid_x']
y_val = data['valid_y']


In [ ]:
train_imgs, train_labels = get_imgs(train_data_dir, train, True)

In [ ]:
#train val split
X_tra, X_val, y_tra, y_val = train_test_split(train_imgs, train_labels, test_size = 3000, random_state = 222)


In [12]:
gc.collect()

378639

In [13]:
X_tra.shape

(9600, 224, 224, 3)

In [ ]:
import pickle

data_save = {}
data_save['train_x'] = X_tra
data_save['train_y'] = y_tra
data_save['valid_x'] = X_val
data_save['valid_y'] = y_val

with open('data.pickle', 'wb') as f:
    pickle.dump(data_save, f, pickle.HIGHEST_PROTOCOL)

In [14]:
train_datagen.fit(X_tra)
# val_datagen.fit(X_val)

In [15]:
val_datagen.fit(X_val)

In [16]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

We're going to train our model with SGD and very low learning rate

In [17]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('densenet_1_layer.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = optimizers.Adam(lr=1e-4)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
model.reset_states()

In [18]:
# batch_size=
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=100,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/100
75/75 [==============================] - 52s 697ms/step - loss: 0.6103 - acc: 0.6662 - fmeasure: 0.5613 - val_loss: 0.5120 - val_acc: 0.7572 - val_fmeasure: 0.6498

Epoch 00001: val_loss improved from inf to 0.51195, saving model to densenet_1_layer.h5
Epoch 2/100
75/75 [==============================] - 43s 571ms/step - loss: 0.4720 - acc: 0.7877 - fmeasure: 0.6934 - val_loss: 0.4359 - val_acc: 0.8069 - val_fmeasure: 0.7182

Epoch 00002: val_loss improved from 0.51195 to 0.43594, saving model to densenet_1_layer.h5
Epoch 3/100
75/75 [==============================] - 44s 583ms/step - loss: 0.4163 - acc: 0.8225 - fmeasure: 0.7441 - val_loss: 0.3936 - val_acc: 0.8308 - val_fmeasure: 0.7543

Epoch 00003: val_loss improved from 0.43594 to 0.39359, saving model to densenet_1_layer.h5
Epoch 4/100
75/75 [==============================] - 43s 576ms/step - loss: 0.3789 - acc: 0.8441 - fmeasure: 0.7768 - val_loss: 0.3654 - val_acc: 0.8446 - val_fmeasure: 0.7759

Epoch 00004: val_los

KeyboardInterrupt: 

In [31]:
model = get_model()

#train only last 20 layer
for layer in model.layers:
    layer.trainable = True

opt = optimizers.Adam(lr=5e-7)
# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
    
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [32]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('densenet_all_layers.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

# model.load_weights('densenet_1_layer.h5')
model.load_weights('densenet_all_layers.h5')


In [33]:
batch_size = 16
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=100,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/100
600/600 [==============================] - 209s 349ms/step - loss: 0.0239 - acc: 0.9942 - fmeasure: 0.9920 - val_loss: 0.0566 - val_acc: 0.9807 - val_fmeasure: 0.9736

Epoch 00001: val_loss improved from inf to 0.05664, saving model to densenet_all_layers.h5
Epoch 2/100
600/600 [==============================] - 178s 297ms/step - loss: 0.0237 - acc: 0.9942 - fmeasure: 0.9920 - val_loss: 0.0570 - val_acc: 0.9805 - val_fmeasure: 0.9733

Epoch 00002: val_loss did not improve
Epoch 3/100
 28/600 [>.............................] - ETA: 2:35 - loss: 0.0238 - acc: 0.9942 - fmeasure: 0.9920

KeyboardInterrupt: 

In [25]:
test_imgs = get_imgs(test_data_dir, test)

  8%|▊         | 426/5400 [00:09<01:55, 42.92it/s]/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 5400/5400 [01:17<00:00, 69.57it/s]


In [26]:
test_datagen.fit(test_imgs)

In [27]:
pred = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

11/11 [==============================] - 33s 3s/step


In [28]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,1,0,1,1,1,0,0,0,...,0,1,0,1,0,0,0,0,1,0
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,0,1,1,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [29]:
sub.to_csv('submission.csv', index=False)

In [30]:
sub.shape

(5400, 86)

## Final Thoughts

This submission should get you around $\approx$0.80 on the LB. and if you've noticed that our last epochs val_fmeasure is the same so it means that our val set is represantion of the test set and you can train for many epochs with EarlyStopping without worring about overfitting on val or train set.

### How to improve from here?
You can change many things which will let you get higher LB score. Following is a small list
<ul>
    <li> Change the Image size to bigger number </li>
    <li> Increse the number of epoch in the fully trainable network($2^{nd}$ training) </li>
    <li> Use diffrent architechure. you'll get more info on that [here](keras.io/applications/)</li>
    <li> If nothing works ensemble is your best friend </li>
</ul>

I'll try to keep improving this notebook. Feel free to contribuite.

Thanks
